In [2]:
import pandas as pd
import requests
import pyarrow.feather as feather

In [3]:
articles_df = pd.read_feather('Data/dataframes/article_dataframe.feather')

In [4]:
article_names = articles_df['article']

# Here fetch a token from hugging face website
# https://huggingface.co/blog/getting-started-with-embeddings
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = "insert token here"

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

In [7]:
def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

article_names_list = article_names.tolist()

#replace _ with space in article names
article_names_list = [name.replace("_", " ") for name in article_names_list]

output = query(article_names_list)

articles_df['embeddings'] = output

articles_df.to_feather('Data/dataframes/article_dataframe_with_embeddings.feather')

In [3]:
# Load data
articles_df = pd.read_feather('Data/dataframes/article_dataframe.feather')
article_names = articles_df['article']

# Define Hugging Face API parameters
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = "insert token here"
api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    # Send a request to Hugging Face API
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options": {"wait_for_model": True}})
    
    # Check for errors in response
    if response.status_code != 200:
        raise Exception(f"Request failed with status code {response.status_code}: {response.text}")
    
    return response.json()

# Prepare article names
article_names_list = article_names.tolist()
article_names_list = [name.replace("_", " ") for name in article_names_list]

# Batch processing to avoid overloading the API
batch_size = 32
embeddings = []

for i in range(0, len(article_names_list), batch_size):
    batch = article_names_list[i:i + batch_size]
    output = query(batch)  # Query embeddings for the batch
    embeddings.extend(output)  # Append the embeddings to the list

# Add embeddings to the DataFrame
articles_df['embeddings'] = embeddings

# Save the updated DataFrame
articles_df.to_feather('Data/dataframes/article_dataframe_with_embeddings.feather')


Exception: Request failed with status code 400: {"error":"Authorization header is correct, but the token seems invalid"}

In [ ]:
articles_df